In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
!echo "HOSTNAME     " `hostname`; 
!echo "CURRENT DIR  " `pwd`

# TAREA: Paraleliza el siguiente código con `omp for reduction`

In [ ]:
%%writefile files/array_reduce.c
/**
  Based on Tim Mattons' tutorial at Intel
  https://software.intel.com/en-us/courseware/249662
**/

#include <stdio.h>
#include <omp.h>
#include <stdlib.h>

#define MAX_THREADS 16
#define SIZE 10000000

int main ()
{
    
    double *a = (double *)malloc(sizeof(double)*SIZE);
    int i,j;
    double start_time, run_time;

    // fill up some values
    for (i=0; i<SIZE; i++) a[i] = 1;

    printf("total secs nthreads\n");

    for (j=1;j<=MAX_THREADS ;j++) {
        omp_set_num_threads(j);
        start_time = omp_get_wtime();


        double total = 0.0;
        for (i=0; i<SIZE; i++) total += a[i];

        run_time = omp_get_wtime() - start_time;

        printf("%f %f %d\n",total, run_time,j);
    }
}

In [ ]:
!make -f m/Makefile-p2.5.1 clean all 

In [ ]:
!./files/array_reduce > files/array_reduce.data
df = pd.read_csv("files/array_reduce.data", sep=" ")
secs     = df['secs']
nthreads = df['nthreads']
total    = df['total']
print np.mean(total)
ideal_secs = secs[0] / nthreads
ax = plt.figure(figsize=(10,5)).add_subplot(111)
ax.plot(nthreads, secs, linewidth=5, alpha=0.5)
ax.plot(nthreads, ideal_secs, linewidth=2, alpha=1., c="black")

# TAREA: Recuerda el código serie para calcular $\pi$. Paralelízalo con `omp for reduction`

In [ ]:
%%writefile files/pi_reduce.c
/**
  Based on Tim Mattons' tutorial at Intel
  https://software.intel.com/en-us/courseware/249662
**/

#include <stdio.h>
#include <omp.h>

#define MAX_THREADS 32

static long num_steps = 100000000;
double step;
int main ()
{
    int i,j;
    double pi, full_sum = 0.0;
    double start_time, run_time;

    step = 1.0/(double) num_steps;

    printf("pi secs nthreads\n");

    j=4;

    for (j=1;j<=MAX_THREADS ;j++) {
        omp_set_num_threads(j);
        full_sum=0.0;
        start_time = omp_get_wtime();

        // --- TU CODIGO AQUI

        run_time = omp_get_wtime() - start_time;
        printf("%f %f %d\n",pi,run_time,j);
    }
}

In [ ]:
!make -f m/Makefile-p2.5.2 clean all > /dev/null

In [ ]:
!./files/pi_reduce > files/pi_reduce.data
df = pd.read_csv("files/pi_reduce.data", sep=" ")
secs     = df['secs']
nthreads = df['nthreads']
pi    = df['pi']
print np.mean(pi)
ideal_secs = secs[0] / nthreads
ax = plt.figure(figsize=(10,5)).add_subplot(111)
ax.plot(nthreads, secs, linewidth=5, alpha=0.5)
ax.plot(nthreads, ideal_secs, linewidth=2, alpha=1., c="black")